In [12]:
import pandas as pd 

recepten =pd.read_csv('data_nutritions.csv')

In [13]:
recepten =pd.read_csv('data_nutritions.csv')

# Kolom voor elke onderdeel
voedingsstoffen = ['energie', 'koolhydraten', 'waarvan suikers', 'natrium', 'eiwit', 'vet', 'waarvan verzadigd', 'vezels']

for stof in voedingsstoffen:
    # Een kolom voor elk stof en zetten op waarde 1 als het hetzelfde is als de stof kolom
    recepten[stof] = (recepten['nutrition'] == stpf).astype(int)

# Merge de kolommen op basis van de nutriëntnaam
for stof in voedingsstoffen:
    recepten_stof = recepten[recepten['nutrition'] == stof]
    waarde = recepten_stof[['value']]
    waarde.columns = [stof]  # Hernoem de kolom naar de juiste stof
    recepten = recepten.merge(waarde, how='left', left_index=True, right_index=True)

# Verwijder de overige kolommen
recepten = recepten.drop(columns=['nutrition', 'value'])

recepten.head(10)  


,Unnamed: 0,recipe,energie_x,koolhydraten_x,waarvan suikers_x,natrium_x,eiwit_x,vet_x,waarvan verzadigd_x,vezels_x,energie_y,koolhydraten_y,waarvan suikers_y,natrium_y,eiwit_y,vet_y,waarvan verzadigd_y,vezels_y
0,0,Kruidnoten met choco-discodip,1,1,1,1,1,1,1,1,260 kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,34 g,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,22 g,NaN,NaN,NaN,NaN,NaN
3,3,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,NaN,200 mg,NaN,NaN,NaN,NaN
4,4,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,3 g,NaN,NaN,NaN
5,5,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,12 g,NaN,NaN
6,6,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7 g,NaN
7,7,Kruidnoten met choco-discodip,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 g
8,8,Kruidnoten in marsepein,1,1,1,1,1,1,1,1,265 kcal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Kruidnoten in marsepein,0,0,0,0,0,0,0,0,NaN,43 g,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
agg_functions = {'energie_y': 'sum', 'koolhydraten_y': 'sum', 'waarvan suikers_y': 'sum', 'natrium_y': 'sum','eiwit_y': 'sum', 'waarvan verzadigd_y': 'sum', 'vezels_y': 'sum'} #rijen die behoren tot één recept samenvoegen
nutri = recepten.groupby(recepten['recipe']).aggregate(agg_functions).reset_index() #de index resetten
nutri.loc[nutri["recipe"] == "Kruidnoten met choco-discodip"] 
nutri

,recipe,energie_y,koolhydraten_y,waarvan suikers_y,natrium_y,eiwit_y,waarvan verzadigd_y,vezels_y
0,Bietensoep met selderijroom,160 kcal,11 g,0,1160 mg,3 g,5 g,5 g
1,Libanese wraps met appel-koolsalade,440 kcal,45 g,0,440 mg,21 g,3 g,9 g
2,Sticky cauliflower (gegrilde bloemkoolroosjes...,235 kcal,16 g,0,800 mg,8 g,2 g,6 g
3,Zalmspies met sojaglaze van de BBQ,300 kcal,17 g,0,560 mg,16 g,3 g,2 g
4,&amp;Suus’ buik-weg-salade,215 kcal,15 g,0,1060 mg,20 g,2 g,7 g
...,...,...,...,...,...,...,...,...
6030,Zweedse gehaktballetjes met room,640 kcal,26 g,0,520 mg,33 g,22 g,2 g
6031,Zweedse vegaballetjes met aardappel &amp; broc...,770 kcal,76 g,0,1320 mg,38 g,10 g,17 g
6032,Zwitserse kaasfondue,850 kcal,7 g,5 g,920 mg,56 g,38 g,0 g
6033,Îles flottantes,290 kcal,49 g,0,125 mg,10 g,3 g,0


In [20]:
energie = nutri['energie_y'].tolist()
energie = [int(i[0:-4]) for i in energie]   

waarde_energie = pd.DataFrame({'energie': energie})
waarde_energie  

nutrix = nutri.merge(waarde_energie, how='left', left_index = True , right_index = True ) 
nutrix

koolhydraten = nutrix['koolhydraten_y'].tolist() 
res = [(sub.split(' ')[0]) for sub in koolhydraten] 
#print(res)
koolhydraten = [int(i[0:-1]) for i in koolhydraten]   

waarde_koolhydraten = pd.DataFrame({'koolhydraten': koolhydraten})
waarde_koolhydraten

# # nutri = nutri.merge(waarde_koolhydraten, how='left', left_index = True , right_index = True )
# # nutri   

# waarvan_suikers = nutri['waarvan suikers'].tolist()
# waarvan_suikers = [int(i[0:-1]) for i in waarvan_suikers]   

# waarde_suikers = pd.DataFrame({'waarvan suikers': waarvan_suikers})
# waarde_suikers

# nutri = nutri.merge(waarde_suikers, how='left', left_index = True , right_index = True )
# nutri   

# natrium = nutri['natrium'].tolist()
# natrium = [int(i[0:-1]) for i in natrium]   

# waarde_natrium  = pd.DataFrame({'natrium': natrium})
# waarde_natrium 

# nutri = nutri.merge(natrium, how='left', left_index = True , right_index = True )
# nutri   

# eiwit = nutri['eiwit'].tolist()
# eiwit = [int(i[0:-1]) for i in eiwit]   

# waarde_eiwit  = pd.DataFrame({'eiwit': eiwit})
# waarde_eiwit 

# nutri = nutri.merge(eiwit, how='left', left_index = True , right_index = True )
# nutri   

# waarvan_verzadigd = nutri['waarvan verzadigd'].tolist()
# waarvan_verzadigd = [int(i[0:-1]) for i in waarvan_verzadigd]   

# waarde_waarvan_verzadigd  = pd.DataFrame({'waarvan verzadigd': waarvan_verzadigd})
# waarde_waarvan_verzadigd

# nutri = nutri.merge(waarde_waarvan_verzadigd, how='left', left_index = True , right_index = True )
# nutri    

# vezels = nutri['vezels_y'].tolist()
# vezels = [int(i[0:-1]) for i in vezels]   

# waarde_vezels = pd.DataFrame({'vezels': vezels})
# waarde_vezels

# nutri = nutri.merge(waarde_vezels, how='left', left_index = True , right_index = True )
# nutri   


# # nutru.drop(columns=['en','Unnamed: 0','nutrition'], inplace = True)  

# # # ingr_alles 

TypeError: 'int' object is not subscriptable

In [5]:
energie = nutri['energie'].tolist()
energie = [int(i[0:-4]) for i in energie]   

waarde_energie = pd.DataFrame({'energie':energie})
waarde_energie 

nutri = nutri.merge(waarde_energie, how='left', left_index = True , right_index = True )
nutri 

NameError: name 'nutri' is not defined

In [6]:
koolhydraten = nutri['koolhydraten'].tolist()
koolhydraten = [int(i[0:-1]) for i in koolhydraten]   

waarde_koolhydraten = pd.DataFrame({'koolhydraten': koolhydraten})
waarde_koolhydraten

nutri = nutri.merge(waarde_koolhydraten, how='left', left_index = True , right_index = True )
nutri   

waarvan_suikers = nutri['waarvan suikers'].tolist()
waarvan_suikers = [int(i[0:-1]) for i in waarvan_suikers]   

waarde_suikers = pd.DataFrame({'waarvan suikers': waarvan_suikers})
waarde_suikers

nutri = nutri.merge(waarde_suikers, how='left', left_index = True , right_index = True )
nutri   

natrium = nutri['natrium'].tolist()
natrium = [int(i[0:-1]) for i in natrium]   

waarde_natrium  = pd.DataFrame({'natrium': natrium})
waarde_natrium 

nutri = nutri.merge(natrium, how='left', left_index = True , right_index = True )
nutri   

eiwit = nutri['eiwit'].tolist()
eiwit = [int(i[0:-1]) for i in eiwit]   

waarde_eiwit  = pd.DataFrame({'eiwit': eiwit})
waarde_eiwit 

nutri = nutri.merge(eiwit, how='left', left_index = True , right_index = True )
nutri   

waarvan_verzadigd = nutri['waarvan verzadigd'].tolist()
waarvan_verzadigd = [int(i[0:-1]) for i in waarvan_verzadigd]   

waarde_waarvan_verzadigd  = pd.DataFrame({'waarvan verzadigd': waarvan_verzadigd})
waarde_waarvan_verzadigd

nutri = nutri.merge(waarde_waarvan_verzadigd, how='left', left_index = True , right_index = True )
nutri    

vezels = nutri['vezels_y'].tolist()
vezels = [int(i[0:-1]) for i in vezels]   

waarde_vezels = pd.DataFrame({'vezels': vezels})
waarde_vezels

nutri = nutri.merge(waarde_vezels, how='left', left_index = True , right_index = True )
nutri  

NameError: name 'nutri' is not defined